<a href="https://colab.research.google.com/github/Pioterr/Arch_linux/blob/master/Saving_Loading_models_tuto_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision
!df -h


In [ ]:
# When it comes to saving and loading models,
# there are three core functions to be familiar with:
###############################################################################
# torch.save: Saves a serialized object to disk.
#     This function uses Python’s pickle utility for serialization.
#     Models, tensors, and dictionaries of all kinds of
#     objects can be saved using this function.
###############################################################################
# torch.load: Uses pickle’s unpickling facilities to deserialize
#     pickled object files to memory. This function also facilitates the
#     device to load the data into (see Saving & Loading Model Across Devices).
###############################################################################
# torch.nn.Module.load_state_dict: Loads a model’s parameter dictionary
#     using a deserialized state_dict. For more information on state_dict.
###############################################################################
# Let’s take a look at the state_dict from the simple model
# used in the Training a classifier tutorial

# What is a ``state_dict``?
# -------------------------
#
# In PyTorch, the learnable parameters (i.e. weights and biases) of an
# ``torch.nn.Module`` model is contained in the model’s *parameters*
# (accessed with ``model.parameters()``). A *state_dict* is simply a
# Python dictionary object that maps each layer to its parameter tensor.
# Note that only layers with learnable parameters (convolutional layers,
# linear layers, etc.) have entries in the model’s *state_dict*. Optimizer
# objects (``torch.optim``) also have a *state_dict*, which contains
# information about the optimizer’s state, as well as the hyperparameters
# used.
#
# Because *state_dict* objects are Python dictionaries, they can be easily
# saved, updated, altered, and restored, adding a great deal of modularity
# to PyTorch models and optimizers.
#
# Example:
# ^^^^^^^^
#
# Let’s take a look at the *state_dict* from the simple model used in the
# `Training a classifier tutorial.

import torch

# Define model



class TheModelClass(torch.nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model
model = TheModelClass()

# Initialize optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])



In [ ]:
####### BOF ########

# SAVING & LOADING MODEL FOR INFERENCE
# Save/Load state_dict (Recommended)

# Save:

torch.save(model.state_dict(), PATH)

# Load:

model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load(PATH))
model.eval()

# When saving a model for inference, it is only necessary to save the trained
# model’s learned parameters. Saving the model’s state_dict with the torch.save()
# function will give you the most flexibility for restoring the model later,
# which is why it is the recommended method for saving models.

# A common PyTorch convention is to save models
# using either a .pt or .pth file extension.

# Remember that you must call model.eval() to set dropout and batch
# normalization layers to evaluation mode before running inference.
# Failing to do this will yield inconsistent inference results.



In [ ]:
####### BOF ########

# Save/Load Entire Model

# Save:

torch.save(model, PATH)

# Load:

# Model class must be defined somewhere
model = torch.load(PATH)
model.eval()

# This save/load process uses the most intuitive syntax and involves
# the least amount of code. Saving a model in this way will save the
# entire module using Python’s pickle module. The disadvantage of this
# approach is that the serialized data is bound to the specific classes
# and the exact directory structure used when the model is saved.
# The reason for this is because pickle does not save the model class itself.
# Rather, it saves a path to the file containing the class, which is used during
# load time. Because of this, your code can break in various ways when used in
# other projects or after refactors.

# A common PyTorch convention is to save models using either a .pt or .pth
# file extension.

# Remember that you must call model.eval() to set dropout and
# batch normalization layers to evaluation mode before running inference.
# Failing to do this will yield inconsistent inference results.

In [ ]:
####### OUI ########

# SAVING & LOADING A GENERAL CHECKPOINT FOR INFERENCE AND/OR RESUMING TRAINING
# Save:
torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            ...
            }, PATH)
# Load:
model = TheModelClass(*args, **kwargs)
optimizer = TheOptimizerClass(*args, **kwargs)

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.eval()
# - or -
model.train()

# When saving a general checkpoint, to be used for either inference or resuming
# training, you must save more than just the model’s state_dict.
# It is important to also save the optimizer’s state_dict, as this contains
# buffers and parameters that are updated as the model trains. Other items that
# you may want to save are the epoch you left off on, the latest recorded
# training loss, external torch.nn.Embedding layers, etc.

# To save multiple components, organize them in a dictionary and use torch.save()
# to serialize the dictionary. A common PyTorch convention is to save
# these checkpoints using the .tar file extension.

# To load the items, first initialize the model and optimizer, then 
# oad the dictionary locally using torch.load(). From here, you can easily
# access the saved items by simply querying the dictionary as you would expect.

# Remember that you must call model.eval() to set dropout and batch normalization
# layers to evaluation mode before running inference. Failing to do this will
# yield inconsistent inference results.
# If you wish to resuming training,
# call model.train() to ensure these layers are in training mode.

In [ ]:
# SAVING MULTIPLE MODELS IN ONE FILE
# Save:
torch.save({
            'modelA_state_dict': modelA.state_dict(),
            'modelB_state_dict': modelB.state_dict(),
            'optimizerA_state_dict': optimizerA.state_dict(),
            'optimizerB_state_dict': optimizerB.state_dict(),
            ...
            }, PATH)
# Load:
modelA = TheModelAClass(*args, **kwargs)
modelB = TheModelBClass(*args, **kwargs)
optimizerA = TheOptimizerAClass(*args, **kwargs)
optimizerB = TheOptimizerBClass(*args, **kwargs)

checkpoint = torch.load(PATH)
modelA.load_state_dict(checkpoint['modelA_state_dict'])
modelB.load_state_dict(checkpoint['modelB_state_dict'])
optimizerA.load_state_dict(checkpoint['optimizerA_state_dict'])
optimizerB.load_state_dict(checkpoint['optimizerB_state_dict'])

modelA.eval()
modelB.eval()
# - or -
modelA.train()
modelB.train()

# When saving a model comprised of multiple torch.nn.Modules, such as a GAN,
# a sequence-to-sequence model, or an ensemble of models, you follow the same
# approach as when you are saving a general checkpoint. In other words, save a
# dictionary of each model’s state_dict and corresponding optimizer.
# As mentioned before, you can save any other items that may aid you in
# resuming training by simply appending them to the dictionary.

# A common PyTorch convention is to save these checkpoints using the .tar
# file extension.

# To load the models, first initialize the models and optimizers,
# then load the dictionary locally using torch.load(). From here, you can easily
# access the saved items by simply querying the dictionary as you would expect.

# Remember that you must call model.eval() to set dropout and batch normalization
# layers to evaluation mode before running inference. Failing to do this will
# yield inconsistent inference results. If you wish to resuming training, call
# model.train() to set these layers to training mode.